In [31]:
import numpy as np
from numpy import ndarray
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [32]:
X, y = make_blobs(500, centers=2, cluster_std=1.4)
y[y == 0] = -1

In [33]:
n_samples, n_features = X.shape
alphas = np.random.randn(n_samples)

In [34]:
LR = 0.1
EPOCHS = 100
C = 1

In [ ]:
def linear_kernel(x1: ndarray, x2: ndarray) -> ndarray:
    return x1@x2

In [35]:
def decision_func(features: ndarray = X) -> ndarray:
    pass

In [36]:
def subj() -> float:
    pass

In [37]:
def get_grad(X: ndarray = X, y: ndarray = y) -> ndarray:
    pass

In [38]:
def fit(batch_size: int = 32):
    for e in range(EPOCHS):
        print(f'Epoch {e}/{EPOCHS}, Loss: {subj()}{20*" "}', end='\r')
        global alphas

        grad_alphas = get_grad()
        alphas -= LR*grad_alphas

In [39]:
def draw_decision_boundary(X: ndarray, y: ndarray):
    xx, yy = np.meshgrid(np.linspace(X[:, 0].min()-1, X[:, 0].max()+1, X.shape[0]),
                         np.linspace(X[:, 1].min()-1, X[:, 1].max()+1, X.shape[0]))
    X_grid = np.c_[xx.ravel(), yy.ravel()]
    pred = decision_func(features=X_grid)
    pred = pred.reshape(xx.shape)

    fig, ax = plt.subplots()
    fig.set_figwidth(12)
    fig.set_figheight(10)
    ax.contour(xx, yy, pred, levels=[-1, 0, 1], linestyles=['--', '-', '--'], colors=['r', 'k', 'r'])
    ax.scatter(X[:, 0], X[:, 1], c=y)
    plt.show()

In [40]:
print(decision_func())

[-111.16074239  293.72547693  269.29812401 -139.41853775  227.9317149
  201.4741298  -204.3581771  -194.43318817  254.8317093   269.63989149
 -168.47343851 -114.0252888  -189.02554108  -61.55011068 -182.1908856
 -121.25412889 -151.85881714  250.01710078 -189.27161214  259.65698021
  241.92050833  226.41768419  307.38702225 -145.58954755 -225.18090269
  242.40980574  299.8524074   218.95499877 -106.34029624 -173.75715405
  -99.29783382 -186.63445661  382.46001192 -104.56352737  280.65005386
  326.92729819  241.37724306 -140.23427337  327.02874873 -131.65854616
 -171.03198608  259.99714816 -193.52373876 -137.10795401 -127.26477079
  290.01971184  186.99148763 -133.09563349  211.7869885   234.1316931
 -115.72635185  374.75219543  249.83492909  268.13092403 -157.01302898
  208.48355388 -188.02070557  252.42309559  216.21551745  207.22601375
  -88.17267062 -170.4053369   229.12218806  322.44794234 -210.08388037
  -90.08377151 -151.4845143  -147.75598392  318.92550487  283.12848521
 -210.626